In [1]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  # 其实没必要，但是看进度条有底
from collections import defaultdict  
import collections
from scipy import stats
from scipy.spatial.distance import cosine # 余弦相似度
import os, math, warnings, math, pickle
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import datetime
warnings.filterwarnings('ignore')

### 目录：
- 序章：
> 压缩内存的函数&定义数据存储的路径：
- C1. 获取相应数据集：
- C2. 召回准备前的工作——前期：
> 1. 用户UserCF协同过滤使用的用户-（物品，时间）字典；
> 2. 物品ItemCF协同过滤使用的物品-（用户，时间）字典；
> 3. 评判和特征工程要用的最后一次点击-记录到每个用户的所有历史点击&最后一次点击记录——预处理算是；
> 4. 冷启动要用的字典特征——为了冷启动阶段召回用，基于文章/物品特性的召回；
> 5. 获取用户对应的历史点击的文章信息，以字典形式存储：
> 6. 最近点击次数最多的topK个文章的索引；召回不足时使用；
- C3. 召回准备前的工作——后期：
> 1. P0.建立多方法召回的信息储存池；user_multi_recall_dict_info  =  {'itemcf_sim_item_recall': {},'usercf_sim_item_recall': {},'cold_start_recall': {}...} 
> 2. P1.基于ItemCF计算文章相似度矩阵：itemcf_sim() ——得到本地缓存结果：itemcf_i2i_sim.pkl，基于ItemCF得到的协同过滤结果；
> 3. P2.基于UserCF计算用户相似度矩阵：usercf_sim() ——得到本地缓存结果：usercf_u2u_sim.pkl，基于UserCF得到的协同过滤结果；
> 4. P3.Item embedding召回——计算Embedding相似度；——得到本地缓存结果：emb_i2i_sim.pkl，基于物品embedding相似性，为了冷启动；
- C4. 召回工作正式启动阶段：
> 1. P1.itemCF 召回：
>> 1. S1.itemCF召回函数：item_based_recommend()；需要之前计算好的itemcf_i2i_sim.pkl
>> 2. S2.进行ItemCF物品召回，并保存结果评估；
user_multi_recall_dict_info['itemcf_sim_item_recall'] = ItemCF_user_recall_items_dict
将ItemCF_user_recall_items_dict ——得到本地缓存结果：itemcf_recall_dict.pkl
> 2. P2.userCF 召回
>> 1. S1.userCF召回函数：user_based_recommend()；需要之前计算好的usercf_u2u_sim.pkl
>> 2. S2.进行userCF物品召回，并保存结果评估；
user_multi_recall_dict_info['usercf_sim_item_recall'] = UserCF_user_recall_items_dict
将UserCF_user_recall_items_dict ——得到本地缓存结果：usercf_u2u2i_recall.pkl
> 3. P3.冷启动召回——先计算Embedding相似度，为了冷启动使用；
>> 1. S1.定义冷启动函数；
>> 2. S2.添加冷启动召回信息；
user_multi_recall_dict_info['cold_start_recall'] = 
将UserCF_user_recall_items_dict ——得到本地缓存结果：cold_start_user_items_dict.pkl
- C5. 多路召回合并！——这里主要是ItemCF召回
> 定义各路权重字典：weight_dict = {'itemcf_sim_item_recall': 2.0,'usercf_sim_item_recall': 1.0}   
> 1. 函数combine_recall_results()
> 2. 基于已有条件只能选取ItemCF召回
> 3. 最终找回结果保存到本地——得到本地缓存结果：final_recall_items_dict.pkl








### 序章——压缩内存的函数

In [13]:
# 节约内存的一个标配函数，一种数据压缩技术
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df


### 序章：定义数据存储的路径：

In [14]:
data_path = './data_path/' 
save_path = './recall_save_path/'  #保存一些计算结果的文件，比如物品、用户相似度
# 做召回评估的一个标志, 如果不进行评估就是直接使用全量数据进行召回
metric_recall = False

# C1.获取相应数据集

- 这里因为是线下，所以只能用训练集
- 函数1：get_all_click_df——获取训练集就是，丢弃掉重复值

In [16]:
def get_all_click_df(data_path = data_path, offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

- 函数2：get_item_info_df——获取文章的基本属性：修改字段名

In [17]:
# 读取文章的基本属性
def get_item_info_df(data_path):
    item_info_df = pd.read_csv(data_path + 'articles.csv')
    # 为了方便与训练集中的click_article_id拼接，需要把article_id修改成click_article_id
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})
    return item_info_df 

- 函数3：get_item_emb_dict——获取文章的向量属性
> 需要注意，这里面是把文章信息里面的每一片的对应向量都以字典的形式存储；
> 保存在本地存储路径的文件；item_content_emb.pkl

In [23]:
def get_item_emb_dict(data_path):
    item_emb_df = pd.read_csv(data_path + '/articles_emb.csv')
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    # 进行归一化
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))
    return item_emb_dict

- 函数4.：min-max 归一化函数


In [19]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

### 开始处理数据
- P1.获取全量的数据集

In [22]:
all_click_df = get_all_click_df(offline=False)

- P2.对时间戳进行归一化,用于在关联规则的时候计算所谓的时间权重；
> 用户的下一次点击文章/物品很受前一次的影响，所谓的时间就近效应；所以方便处理权重问题；

In [21]:
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)
all_click_df

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,157507,1.000000,4,1,17,1,25,2
1,1,289197,1.000000,4,1,17,1,25,6
2,2,168401,1.000000,4,3,20,1,25,2
3,3,50644,1.000000,4,3,2,1,25,2
4,4,42567,1.000000,4,1,12,1,16,1
...,...,...,...,...,...,...,...,...,...
518005,221924,70758,0.411803,4,3,2,1,25,2
518006,207823,331116,0.411875,4,3,2,1,25,1
518007,207823,234481,0.411976,4,3,2,1,25,1
518008,207823,211442,0.412088,4,3,2,1,25,1


- P3.获取文章信息和文章向量；

In [24]:
item_info_df = get_item_info_df(data_path)
item_emb_dict = get_item_emb_dict(data_path)

- 得到的变量：
> 1. all_click_df：获取全量训练集，用户点击日志，如时间，环境系统地区等；将原click_timestamp标准化，并且文章信息item_info_df关联上；
> 2. item_info_df：文章的基本信息，ID，品类，创建时间，词数；
> 3. item_emb_dict：文章的向量信息


# C2.召回准备的工作——前期；

### S1.用户UserCF协同过滤使用的用户-（物品，时间）字典
- 函数1：get_user_item_time_dict()
- 根据 点击时间 获取用户文章，形成字典 {user1: [(item1, time1), (item2, time2)..]...} —— UserCF
- 基于用户协同过滤使用

In [26]:
# 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
def get_user_item_time_dict(click_df):
    click_df = click_df.sort_values('click_timestamp')
    # 定义一个打包函数
    def make_item_time_pair(df):  
        return list(zip(df['click_article_id'], df['click_timestamp']))
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    return user_item_time_dict

### S2.物品ItemCF协同过滤使用的物品-（用户，时间）字典
- 函数2.get_item_user_time_dict()
- 根据 点击时间 获取商品被点击的用户序列 {item1: [(user1, time1), (user2, time2)...]...} —— ItemCF
- 文章协同过滤时使用

In [27]:
# 根据时间获取商品被点击的用户序列  {item1: [(user1, time1), (user2, time2)...]...}
def get_item_user_time_dict(click_df):
    # 定义一个打包函数
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))
    click_df = click_df.sort_values('click_timestamp')
    item_user_time_df = click_df.groupby('click_article_id')['user_id', 'click_timestamp'].apply(lambda x: make_user_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'user_time_list'})
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df['user_time_list']))
    return item_user_time_dict

### S3.评判和特征工程要用的最后一次点击-记录到每个用户的所有历史点击&最后一次点击记录
- 函数：get_hist_and_last_click()
- 获取当前数据的历史点击和最后一次点击；
- 因为这次项目的评判就是最后一次点击；所以后续特征工程，评判也会用到；

In [28]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]
    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)
    return click_hist_df, click_last_df
# 提取最后一次点击作为召回评估，如果不需要做召回评估直接使用全量的训练集进行召回(线下验证模型)
# 如果不是召回评估，直接使用全量数据进行召回，不用将最后一次提取出来
trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)

### S4.冷启动要用的字典特征——为了冷启动阶段召回用，基于文章/物品特性的召回；
- 函数：get_item_info_dict()
- 获取文章id对应的基本属性，保存成字典的形式，方便后面召回阶段，冷启动阶段直接使用；
- 返回三个对象，一个是文章——类型字典；一个是文章——词数字典；一个是文章——创建时间字典
- 必须基于文章属性获取的情况下才能运行；——这句话看似是废话，但是确实是废话，文章属性没有冷启动启动毛？

In [29]:
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))
    return item_type_dict, item_words_dict, item_created_time_dict
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)
# item_type_dict：每一篇文章的类型字典；
# item_words_dict：每一篇文章的字数字典；
# item_created_time_dict：创建时间信息

### S5.获取用户对应的历史点击的文章信息，以字典形式存储：
- 函数：get_user_hist_item_info_dict()
- 返回四个字典：一是用户和点击文章的类型字典；二是用户和点击文章的字典；三是用户和点击文章平均词语数字典；四是用户和最后一次点击文章时间字典
- 相当于间接做一个简略版的用户画像

In [30]:
def get_user_hist_item_info_dict(all_click):
    # 获取user_id对应的用户历史点击文章类型的集合字典
    user_hist_item_typs = all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict = dict(zip(user_hist_item_typs['user_id'], user_hist_item_typs['category_id']))
    # 获取user_id对应的用户点击文章的集合
    user_hist_item_ids_dict = all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids_dict['user_id'], user_hist_item_ids_dict['click_article_id']))
    # 获取user_id对应的用户历史点击的文章的平均字数字典
    user_hist_item_words = all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))
    # 获取user_id对应的用户最后一次点击的文章的创建时间
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                                user_last_item_created_time['created_at_ts']))
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict

### S6.最近点击次数最多的topK个文章的索引；召回不足时使用；
- 函数：get_item_topk_click()

In [31]:
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

- 得到的变量：
> 1. item_type_dict：每一篇文章的类型字典；
> 2. item_words_dict：每一篇文章的字数字典；
> 3. item_created_time_dict：创建时间信息
> 4. trn_hist_click_df：用户历史点击信息
> 5. trn_last_click_df：用户最后一次点击信息

# C3.召回准备的工作——后期

- S1.建立多方法召回的信息储存池；user_multi_recall_info_dict


In [32]:
# 定义一个多路召回的字典，将各路召回的结果都保存在这个字典当中
user_multi_recall_dict_info =  {'itemcf_sim_item_recall': {}}      
# user_multi_recall_dict_info =  {'itemcf_sim_item_recall': {}, 'usercf_sim_item_recall': {}}    
# user_multi_recall_dict_info =  {'itemcf_sim_itemcf_recall': {},
#                            'embedding_sim_item_recall': {},
#                            'cold_start_recall': {}}            

- S2.召回效果评估函数：metrics_recall()

In [33]:
# - 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率可以
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=5):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict)
    for k in range(10, topk+1, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # 获取前k个召回的结果
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

### P1.基于ItemCF计算文章相似度矩阵：
KDD2020论坛中有关于修正商品偏好修正推荐的逻辑，在计算item2item相似性矩阵时，使用关联规则，使得计算的文章相似性同时需要考虑:
> 1. 用户点击的时间权重
> 2. 文章创建的时间权重
- 两次点击时间差得少，1/exp(0.6 * (1 - 1)) = 1；两次点击时间差得多，1/exp(0.6 * (1 - 0.5)) = 1.34
- 因此需要传入训练数据 all_click_df + item_created_time_dict，后者是为了在文章创建时间上进行权重修正；
- 计算好的文章之间相似度保存在i2i_sim变量，同时生成字典存在本地itemcf_i2i_sim.pkl，可以后续加载
- 函数：itemcf_sim()

In [34]:
def itemcf_sim(all_click_df, item_created_time_dict):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    # S1.先获取用户的每一篇文章，以及点击的时间，以字典形式
    user_item_time_dict = get_user_item_time_dict(all_click_df)
    # 计算物品相似度
    #---下面为余弦相似度计算公式的分子部分；这里是点没点，也就是0-1所以可以用余弦相似度/杰卡德相似度
    i2i_sim = {} #记录物品之间的相似度字典——准确来说是相似度的分子部分
    item_cnt = defaultdict(int) # 记录相似物品数量的字典；defaultdict可以应对key值不存在的情形，不然的话还得用if else来判断它是否存在，不存在新建，存在才在下面+=1
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间 + 物品流行度等因素；这里暂时没有考虑流行度；
        # 还要遍历每个用户所有的点击文章和时间，其实就是所有的物品和时间，i是物品，click_time是触发时间
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})#setdefault其实就是初始化，如果已经存在了i就保持不变，如果不存在就初始化一个出来
            for loc2, (j, j_click_time) in enumerate(item_time_list): # 物品与物品的相似度，当然还得遍历一遍物品 
                if(i == j):   # 物品与物品之间相同，直接跳过
                    continue      
                # 考虑文章的正向顺序点击和反向顺序点击    
                loc_alpha = 1.0 if loc2 > loc1 else 0.7 
                # 位置信息权重，其中的参数可以调节
                loc_weight = loc_alpha * (0.8 ** (np.abs(loc2 - loc1) - 1))
                # 点击时间权重，其中的参数可以调节
                click_time_weight = np.exp(0.8 ** np.abs(i_click_time - j_click_time))
                # 两篇文章创建时间的权重，其中的参数可以调节
                created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                i2i_sim[i].setdefault(j, 0)
                # 考虑多种因素的权重计算最终的文章之间的相似度
                i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1) 
#                 i2i_sim[i][j] += click_time_weight * created_time_weight / math.log(len(item_time_list) + 1) 
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    # 将得到的相似性矩阵保存到本地，保存到指定路径的itemcf_i2i_sim.pkl
    # 也可以保存成csv吧？
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    return i2i_sim_
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict)
# 已经计算完成的话可以直接读取本地的存储文件
# i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

100%|█████████████████████████████████████████████████████████████████████████| 247496/247496 [04:55<00:00, 837.71it/s]


## P2.基于UserCF计算用户相似度矩阵：
- 在计算用户之间的相似度的时候，也可以使用一些简单的关联规则，
- 比如用户活跃度权重，这里将用户的点击次数作为用户活跃度的指标；这里就是记录条数；
- 用二人之间活跃度作权重修正；
- 函数1：get_user_activate_degree_dict()计算活跃度；——但是这种活跃度计算效率不敢恭维。。。谨慎运行，本人已崩N次。。。
- 函数2：usercf_sim()计算用户相似性
> 1. 先用get_user_activate_degree_dict通过用户的点击次数标准化所谓的活跃度；得到一个字典；{用户i:活跃度i。。。。}
> 2. 再用usercf_sim通过用户的点击次数标准化所谓的活跃度；
- 这里由于是采用了采样的办法，所以有一部分用户字典中没有键值，只有键；
- 计算好的用户相似型信息保存在本地存储路径文件：usercf_u2u_sim.pkl

In [35]:
def get_user_activate_degree_dict(all_click_df):
    all_click_df_ = all_click_df.groupby('user_id')['click_article_id'].count().reset_index()
    # 用户活跃度归一化
    mm = MinMaxScaler()
    all_click_df_['click_article_id'] = mm.fit_transform(all_click_df_[['click_article_id']])
    user_activate_degree_dict = dict(zip(all_click_df_['user_id'], all_click_df_['click_article_id']))
    return user_activate_degree_dict

In [22]:
def usercf_sim(all_click_df, user_activate_degree_dict):
    """
        用户相似性矩阵计算
        :param all_click_df: 数据表
        :param user_activate_degree_dict: 用户活跃度的字典
        return 用户相似性矩阵
        
        思路: 基于用户的协同过滤(详细请参考上一期推荐系统基础的组队学习) + 关联规则
    """
    item_user_time_dict = get_item_user_time_dict(all_click_df)
    
    u2u_sim = {}
    user_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, click_time in user_time_list:
            user_cnt[u] += 1
            u2u_sim.setdefault(u, {})
            for v, click_time in user_time_list:
                u2u_sim[u].setdefault(v, 0)
                if u == v:
                    continue
                # 用户平均活跃度作为活跃度的权重，这里的式子也可以改善
                activate_weight = 100 * 0.5 * (user_activate_degree_dict[u] + user_activate_degree_dict[v])   
                u2u_sim[u][v] += activate_weight / math.log(len(user_time_list) + 1)
    u2u_sim_ = u2u_sim.copy()
    for u, related_users in u2u_sim.items():
        for v, wij in related_users.items():
            u2u_sim_[u][v] = wij / math.sqrt(user_cnt[u] * user_cnt[v])
    # 将得到的相似性矩阵保存到本地
    pickle.dump(u2u_sim_, open(save_path + 'usercf_u2u_sim.pkl', 'wb'))
    return u2u_sim_
user_activate_degree_dict = get_user_activate_degree_dict(all_click_df)
# u2u_sim = usercf_sim(all_click_df, user_activate_degree_dict)
# 已经计算好的可以直接读取本地文件
u2u_sim = pickle.load(open(save_path + 'usercf_u2u_sim.pkl', 'rb'))

## P3.Item embedding召回——计算Embedding相似度；

### S1.事前准备
- 有个思路：
> 1. 结合用户的行为，比如他点击最多的品类；—因为还是从物品的角度考虑，所以叫ItemEmbedding基于计算好的emb相似度，去召回；
> 2. 拼接上topN点击数最多的文章
- 以下代码慎重运行；

In [ ]:
def cosine_similarity_def(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / denom
i2i_emb_sim = {}
for item_i in tqdm(list(item_emb_dict.keys())):
    i2i_emb_sim.setdefault(item_i, {})
    for item_j in item_emb_dict:
        if item_i == item_j :
            continue
        i2i_emb_sim[item_i].setdefault(item_j, 0)
        similarity_ij = cosine_similarity_def(item_emb_dict[item_i], item_emb_dict[item_j])
        if similarity_ij>=0.8:
            i2i_emb_sim[item_i][item_j] = similarity_ij
        else:
            continue
pickle.dump(i2i_emb_sim, open(save_path + 'emb_i2i_sim.pkl', 'wb'))
# 如果有直接计算好的可以直接读取
emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

# C4.召回正式流程启动！

## P1.itemCF 召回

### S1.定义Item召回函数
- 函数：item_based_recommend()
- 需要
> 1. 指定用户；                      ——需要遍历：
> 2. 用户的点击物品-时间字典；            ——函数get_user_item_time_dict得到，user_item_time_dict
> 3. 物品相似度矩阵：                  ——函数itemcf_sim得到，i2i_sim
> 4. 返回的相似物品数目；               ——需要指定sim_item_topk
> 5. 需要召回的物品数目；               ——需要指定recall_item_num              
> 6. 最热门的点击物品（补足用）；          ——通过函数get_item_topk_click()得到，
> 7. 物品创建时间字典 ；                ——item_created_time_dict

In [36]:
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict):    
    """
        基于文章协同过滤的召回
        : user_id: 用户id
        : user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列  {user1: [(item1, time1), (item2, time2)..]...}
        : i2i_sim: 字典，物品文章相似性矩阵
        : sim_item_topk: 整数， 选择与当前文章物品最相似的前k篇文章
        : recall_item_num: 整数， 最后的召回文章数量
        : item_topk_click: 列表，点击次数最多的文章列表，用户召回补全
        : emb_i2i_sim: 字典基于内容embedding算的文章相似矩阵
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items }
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
#         try:
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            # 文章创建时间差权重加入到其中尝试
            created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            content_weight = 1.0
            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * content_weight * wij
#         except:
#             continue
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
    return item_rank

## S2.进行ItemCF物品召回，并保存结果评估；

In [37]:
# 先进行itemcf召回, 为了召回评估，所以提取最后一次点击
# 这里走的是下面这一条，就是历史点击信息，也就是全训练数据
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df
# 默认生成一个字典，保存后续每一种召回方式召回的结果
# user_recall_items_dict = collections.defaultdict(dict)
ItemCF_user_recall_items_dict = collections.defaultdict(dict)
# 得到用户点击历史：即用户u:([物品i,点击时间i]...)字典
user_item_time_dict = get_user_item_time_dict(trn_hist_click_df)
# 得到物品间的相似度，加载本地得到的结果，是一个字典
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
# 用50个点击数最多的文章做备用填充
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)
# 指定相似物品数30,召回物品数20
sim_item_topk = 30
recall_item_num = 20

#### WARNING：如果物品召回数据信息ItemCF_user_recall_items_dict，已经计算好的话，下面这段代码要跳过，太吃内存；

In [38]:
# 遍历训练集中所有的去重用户：
# 参数是指定的用户id，用户点击物品时间字典，物品相似度，需要返回的相似物品数目，需要召回的物品数，做备用的50个点击数最多的文章物品，文章物品创建时间
for user_id in tqdm(trn_hist_click_df['user_id'].unique()):
#     user_recall_items_dict[user_id] = item_based_recommend(user_id, user_item_time_dict, \
#                                                         i2i_sim, sim_item_topk, recall_item_num, \
#                                                         item_topk_click, item_created_time_dict)
    ItemCF_user_recall_items_dict[user_id] = item_based_recommend(user_id, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict)
user_multi_recall_dict_info['itemcf_sim_item_recall'] = ItemCF_user_recall_items_dict
# 基于用户ItemCF的召回结果保存到本地指定路径文件名为itemcf_recall_dict.pkl
pickle.dump(user_multi_recall_dict_info['itemcf_sim_item_recall'], open(save_path + 'itemcf_recall_dict.pkl', 'wb'))    

100%|████████████████████████████████████████████████████████████████████████| 247496/247496 [1:30:25<00:00, 45.61it/s]


### 已经计算好的ItemCF召回结果可以直接读取

In [39]:
# 已经计算好的ItemCF召回结果可以直接读取
ItemCF_user_recall_items_dict = pickle.load(open(save_path + 'itemcf_recall_dict.pkl', 'rb'))
user_multi_recall_dict_info['itemcf_sim_item_recall'] = ItemCF_user_recall_items_dict
# # 如果需要评估：
# if metric_recall:
#     # 召回效果评估
#     metrics_recall(user_multi_recall_dict_info['itemcf_sim_itemcf_recall'], trn_last_click_df, topk=recall_item_num)

## P2.基于userCF召回；

### S1.定义User召回函数
- 函数：user_based_recommend()
- 需要
> 1. 指定用户；                      ——需要遍历：
> 2. 用户的点击物品-时间字典；            ——函数get_user_item_time_dict得到，user_item_time_dict
> 3. 用户相似度矩阵：                  ——函数usercf_sim得到或者加载本地储存结果，u2u_sim
> 4. 返回的相似用户数目；               ——需要指定sim_user_topk
> 5. 需要召回的物品数目；               ——需要指定recall_item_num              
> 6. 最热门的点击物品（补足用）；          ——通过函数get_item_topk_click()得到，
> 7. 物品创建时间字典 ；                ——item_created_time_dict

In [40]:
# 基于用户的召回 u2u2i
def user_based_recommend(user_id, user_item_time_dict, u2u_sim, sim_user_topk, recall_item_num, item_topk_click, item_created_time_dict):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param u2u_sim: 字典，用户间相似度矩阵
        :param sim_user_topk: 整数， 选择与当前用户最相似的前k个用户
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，召回补全
        :param item_created_time_dict: 文章创建时间列表
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 历史交互
    user_item_time_list = user_item_time_dict[user_id]    # {item1: time1, item2: time2...}
    user_hist_items = set([i for i, t in user_item_time_list])   # 存在一个用户与某篇文章的多次交互， 这里得去重
    items_rank = {}
    for sim_u, wuv in sorted(u2u_sim[user_id].items(), key=lambda x: x[1], reverse=True)[:sim_user_topk]:
        for i, click_time in user_item_time_dict[sim_u]:
            if i in user_hist_items:
                continue
            items_rank.setdefault(i, 0)
#             loc_weight = 1.0
            content_weight = 1.0
            created_time_weight = 1.0
            # 当前文章与该用户看的历史文章进行一个权重交互
            for loc, (j, click_time) in enumerate(user_item_time_list):
                # 点击时的相对位置权重
#                 loc_weight += 0.6 ** (len(user_item_time_list) - loc)
                # 内容相似性权重，还是省略掉
#                 if emb_i2i_sim.get(i, {}).get(j, None) is not None:
#                     content_weight += emb_i2i_sim[i][j]
#                 if emb_i2i_sim.get(j, {}).get(i, None) is not None:
#                     content_weight += emb_i2i_sim[j][i]
                # 创建时间差权重
                created_time_weight += np.exp(0.8 * np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            items_rank[i] += content_weight * created_time_weight * wuv
    # 热度补全
    if len(items_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in items_rank.items(): # 填充的item应该不在原来的列表中
                continue
            items_rank[item] = - i - 100 # 随便给个复数就行
            if len(items_rank) == recall_item_num:
                break
    items_rank = sorted(items_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]    
    return items_rank

### S2.进行UserCF物品召回，并保存结果评估；
—— 其实也要慎重运行；

In [ ]:
# 这里是为了召回评估，所以提取最后一次点击
# 由于usercf中计算user之间的相似度的过程太费内存了，全量数据这里就没有跑，跑了一个采样之后的数据
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df
# 生成召回字典
UserCF_user_recall_items_dict = collections.defaultdict(dict)
# 获取用户行为数据字典
user_item_time_dict = get_user_item_time_dict(trn_hist_click_df)
# 加载计算好的用户相似度
u2u_sim = pickle.load(open(save_path + 'usercf_u2u_sim.pkl', 'rb'))

In [ ]:
sim_user_topk = 15
recall_item_num = 10
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)
# 遍历用户，因为有空的用户，所以遍历时按照异常处理
for user in tqdm(trn_hist_click_df['user_id'].unique()):
    try:
        UserCF_user_recall_items_dict[user] = user_based_recommend(user, user_item_time_dict, u2u_sim, sim_user_topk, \
                                                            recall_item_num, item_topk_click, item_created_time_dict)  
    except:
        UserCF_user_recall_items_dict[user] = None
# 用户的召回结果保存到本地指定临时存储路径文件 usercf_u2u2i_recall.pkl
pickle.dump(UserCF_user_recall_items_dict, open(save_path + 'usercf_u2u2i_recall.pkl', 'wb'))
user_multi_recall_dict_info['usercf_sim_item_recall'] = UserCF_user_recall_items_dict
if metric_recall:
    # 召回效果评估
    metrics_recall(user_multi_recall_dict_info, trn_last_click_df, topk=recall_item_num)

### 已经计算好的UserCF召回结果可以直接读取

In [ ]:

UserCF_user_recall_items_dict = pickle.load(open(save_path + 'usercf_u2u2i_recall.pkl', 'rb'))
user_multi_recall_dict_info['usercf_sim_item_recall'] = UserCF_user_recall_items_dict
# # 如果需要评估：
# if metric_recall:
#     # 召回效果评估
#     metrics_recall(user_multi_recall_dict_info['itemcf_sim_itemcf_recall'], trn_last_click_df, topk=recall_item_num)

## P3.冷启动召回——尝试篇——暂时搁浅。。。


- 冷启动问题可以分成三类：文章冷启动-物品向，用户冷启动，系统冷启动。
> 1. 文章冷启动：对于一个平台系统新加入的文章，该文章没有任何的交互记录，如何推荐给用户的问题。
(对于我们场景可以认为是，日志数据中没有出现过的文章都可以认为是冷启动的文章)
> 2. 用户冷启动：对于一个平台系统新来的用户，该用户还没有文章的交互信息，如何给该用户进行推荐。(对于我们场景就是，测试集中的用户是否在测试集对应的log数据中出现过，如果没有出现过，那么可以认为该用户是冷启动用户。但是有时候并没有这么严格，我们也可以自己设定某些指标来判别哪些用户是冷启动用户，比如通过使用时长，点击率，留存率等等)
> 3. 系统冷启动：就是对于一个平台刚上线，还没有任何的相关历史数据，此时就是系统冷启动，其实也就是前面两种的一个综合。
- 鉴于用户冷启动的相似度计算有缺失数据，所以这里逼不得已使用物品冷启动；就是文章冷启动

### S1.定义冷启动召回函数
- 物品没有曝被该用户点击过，但是和用户的行为接触过的很相像的，基于embedding

In [102]:
user_article_click_prefer = all_click_df.merge(item_info_df,left_on="click_article_id",right_on="click_article_id",how="left")
user_article_click_prefer_count = user_article_click_prefer.groupby(["user_id","category_id"])["click_timestamp"].count().reset_index().sort_values(["user_id","click_timestamp"])
user_article_click_prefer_wordsmean = user_article_click_prefer.groupby(["user_id","category_id"])["words_count"].mean().reset_index()
user_article_click_prefer_cold_system = user_article_click_prefer_count.merge(user_article_click_prefer_wordsmean,left_on=["user_id","category_id"],right_on=["user_id","category_id"],how="inner")

((1102431, 3), (1102431, 3))

In [ ]:
# 基于规则进行文章过滤
# 保留文章主题与用户历史浏览主题相似的文章
# 保留文章字数与用户历史浏览文章字数相差不大的文章
# 保留最后一次点击当天的文章
# 按照相似度返回最终的结果
def get_click_article_ids_set(all_click_df):
    return set(all_click_df.click_article_id.values)

def cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                     user_last_item_created_time_dict, item_type_dict, item_words_dict, 
                     item_created_time_dict, click_article_ids_set, recall_item_num):
    """
        冷启动的情况下召回一些文章
        :param user_recall_items_dict: 基于内容embedding相似性召回来的很多文章， 字典， {user1: [item1, item2, ..], }
        :param user_hist_item_typs_dict: 字典， 用户点击的文章的主题映射
        :param user_hist_item_words_dict: 字典， 用户点击的历史文章的字数映射
        :param user_last_item_created_time_idct: 字典，用户点击的历史文章创建时间映射
        :param item_tpye_idct: 字典，文章主题映射
        :param item_words_dict: 字典，文章字数映射
        :param item_created_time_dict: 字典， 文章创建时间映射
        :param click_article_ids_set: 集合，用户点击过得文章, 也就是日志里面出现过的文章
        :param recall_item_num: 召回文章的数量， 这个指的是没有出现在日志里面的文章数量
    """
    cold_start_user_items_dict = {}
    for user, item_list in tqdm(user_recall_items_dict.items()):
        cold_start_user_items_dict.setdefault(user, [])
        for item, score in item_list:
            # 获取历史文章信息
            hist_item_type_set = user_hist_item_typs_dict[user]
            hist_mean_words = user_hist_item_words_dict[user]
            hist_last_item_created_time = user_last_item_created_time_dict[user]
            hist_last_item_created_time = datetime.fromtimestamp(hist_last_item_created_time)
            
            # 获取当前召回文章的信息
            curr_item_type = item_type_dict[item]
            curr_item_words = item_words_dict[item]
            curr_item_created_time = item_created_time_dict[item]
            curr_item_created_time = datetime.fromtimestamp(curr_item_created_time)

            # 首先，文章不能出现在用户的历史点击中， 然后根据文章主题，文章单词数，文章创建时间进行筛选
            if curr_item_type not in hist_item_type_set or \
                item in click_article_ids_set or \
                abs(curr_item_words - hist_mean_words) > 200 or \
                abs((curr_item_created_time - hist_last_item_created_time).days) > 90: 
                continue
                
            cold_start_user_items_dict[user].append((item, score))      # {user1: [(item1, score1), (item2, score2)..]...}
    
    # 需要控制一下冷启动召回的数量
    cold_start_user_items_dict = {k: sorted(v, key=lambda x:x[1], reverse=True)[:recall_item_num] \
                                  for k, v in cold_start_user_items_dict.items()}
    pickle.dump(cold_start_user_items_dict, open(save_path + 'cold_start_user_items_dict.pkl', 'wb'))
    return cold_start_user_items_dict

In [ ]:
# all_click_df_1 = all_click_df.copy()
# # all_click_df_ = all_click_df_.merge(item_info_df, how='left', on='click_article_id')
# user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict = get_user_hist_item_info_dict(all_click_df_1)
# click_article_ids_set = get_click_article_ids_set(all_click_df_1)
# # 需要注意的是
# # 这里使用了很多规则来筛选冷启动的文章，所以前面再召回的阶段就应该尽可能的多召回一些文章，否则很容易被删掉
# cold_start_user_items_dict = cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, 
#                                               user_last_item_created_time_dict, item_type_dict, item_words_dict, 
#                                               item_created_time_dict, click_article_ids_set, recall_item_num)
# user_multi_recall_dict['cold_start_recall'] = cold_start_user_items_dict 


# C5.多路召回合并！——这里主要是ItemCF召回

In [41]:
def combine_recall_results(user_multi_recall_dict, weight_dict=None, topk=25):
    final_recall_items_dict = {}
    # 对每一种召回结果按照用户进行归一化，方便后面多种召回结果，相同用户的物品之间权重相加
    def norm_user_recall_items_sim(sorted_item_list):
        # 如果冷启动中没有文章或者只有一篇文章，直接返回，出现这种情况的原因可能是冷启动召回的文章数量太少了，
        # 基于规则筛选之后就没有文章了, 这里还可以做一些其他的策略性的筛选
        if len(sorted_item_list) < 2:
            return sorted_item_list
        min_sim = sorted_item_list[-1][1]
        max_sim = sorted_item_list[0][1]
        norm_sorted_item_list = []
        for item, score in sorted_item_list:
            if max_sim > 0:
                norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
            else:
                norm_score = 0.0
            norm_sorted_item_list.append((item, norm_score))
        return norm_sorted_item_list
    print('多路召回合并...')
    for method, user_recall_items in tqdm(user_multi_recall_dict.items()):
        print(method + '...')
        # 在计算最终召回结果的时候，也可以为每一种召回结果设置一个权重
        if weight_dict == None:
            recall_method_weight = 1
        else:
            recall_method_weight = weight_dict[method]
        for user_id, sorted_item_list in user_recall_items.items(): # 进行归一化
            user_recall_items[user_id] = norm_user_recall_items_sim(sorted_item_list)
        for user_id, sorted_item_list in user_recall_items.items():
            # print('user_id')
            final_recall_items_dict.setdefault(user_id, {})
            for item, score in sorted_item_list:
                final_recall_items_dict[user_id].setdefault(item, 0)
                final_recall_items_dict[user_id][item] += recall_method_weight * score  
    final_recall_items_dict_rank = {}
    # 多路召回时也可以控制最终的召回数量
    for user, recall_item_dict in final_recall_items_dict.items():
        final_recall_items_dict_rank[user] = sorted(recall_item_dict.items(), key=lambda x: x[1], reverse=True)[:topk]
    # 将多路召回后的最终结果字典保存到本地指定路径final_recall_items_dict.pkl
    pickle.dump(final_recall_items_dict, open(os.path.join(save_path, 'final_recall_items_dict.pkl'),'wb'))
    return final_recall_items_dict_rank 

In [42]:
# 这里直接对多路召回的权重给了一个相同的值，其实可以根据前面召回的情况来调整参数的值；电商往往更偏爱于用ItemCF，所以权重加大一些；
# 保存到本地final_recall_items_dict.pkl
weight_dict = {'itemcf_sim_item_recall': 2.0,'usercf_sim_item_recall': 1.0}    
# weight_dict = {'itemcf_sim_itemcf_recall': 2.0,'embedding_sim_item_recall': 1.0,'cold_start_recall': 1.0}    
# 最终合并之后每个用户召回150个商品进行排序
final_recall_items_dict_rank = combine_recall_results(user_multi_recall_dict_info, weight_dict, topk=150)

多路召回合并...


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

itemcf_sim_item_recall...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.71s/it]


In [43]:
final_recall_items_dict_rank

{0: [(225446, 2.0),
  (3244, 1.2912699376851857),
  (119402, 0.4747491855660218),
  (355172, 0.32934353353173323),
  (182260, 0.2758719433739583),
  (62446, 0.2707973533827773),
  (73351, 0.2571224315746646),
  (331775, 0.2522501253844662),
  (36162, 0.24900326217654942),
  (16123, 0.22893386229554671),
  (207141, 0.19621703534914917),
  (285642, 0.19174536528583266),
  (194341, 0.16725933875185986),
  (73812, 0.15966185865574456),
  (181475, 0.1584815320188587),
  (284220, 0.15423270363194772),
  (203336, 0.12479681250092191),
  (285869, 0.05312045095647498),
  (299499, 0.02390716287519218),
  (211442, 0.0)],
 1: [(30408, 2.0),
  (48401, 1.3424247342480866),
  (308117, 0.712997329356449),
  (298684, 0.5152956310591698),
  (71397, 0.506727939206115),
  (168841, 0.23396113137272834),
  (156279, 0.21881879584161906),
  (63760, 0.21851338975306694),
  (63809, 0.18248453548452),
  (63672, 0.14715935539126937),
  (63783, 0.09558795603278658),
  (327646, 0.07158933582919963),
  (288827, 0.05

In [134]:
i2i_sim[2022][299697]

0.014336927659788359

In [108]:
i2i_sim[31118]

{69778: 0.2594537037504562,
 293062: 0.00014180981317156455,
 293050: 0.0024114528730996864,
 284985: 0.00012918295769662744,
 284422: 0.00026613491576864815,
 336801: 0.0007726543095995243,
 312415: 0.005525227339668959,
 123909: 0.0001867375029796463,
 123757: 0.0017075840877995786,
 123613: 0.0009705257878460476,
 107013: 0.0022425648497133794,
 118391: 0.0019834122691160263,
 124749: 0.001726128140739772,
 140803: 0.008782112343579859,
 58619: 0.0026865205808832894,
 58235: 0.006904754361630691,
 30360: 0.011666477239078047,
 25325: 0.004995100387754111,
 32544: 0.0843270001059344,
 32160: 0.08084482086972519,
 30730: 0.017286163267912213,
 31189: 0.14436575155308073,
 31691: 0.08166560151542061,
 95688: 0.0278578635915192,
 256756: 0.046475532018571936,
 245161: 0.0053704365371093354,
 271677: 0.011223675459217767,
 187065: 0.04520126790040161,
 199474: 0.002241649265070514,
 199197: 0.009899521512262635,
 199696: 0.01509800288541846,
 198613: 0.004636728920158004,
 198144: 0.0013

In [109]:
i2i_sim[187065]

{293301: 0.017714640916935497,
 96877: 0.004157721660477408,
 96798: 0.009029490293564674,
 124749: 0.009235213062460073,
 96418: 0.010542031326572353,
 214800: 0.062704856270601,
 226496: 0.006303122918983368,
 208048: 0.012309013666090763,
 293050: 0.0069758571674742665,
 276970: 0.014185314212924445,
 336245: 0.005380159593733149,
 338340: 0.007490534185847921,
 337143: 0.002700454684952925,
 293513: 0.00386261615321212,
 313996: 0.004847929477421405,
 50504: 0.0023340994189578047,
 58619: 0.0015488422738106254,
 58891: 0.011016589004652192,
 29953: 0.004447503601703344,
 32750: 0.018381400976411034,
 96210: 0.019772063795957112,
 215861: 0.016507248138517058,
 242596: 0.012557843342483761,
 270607: 0.01126483984772002,
 271262: 0.011159135176065255,
 168623: 0.046050618666123984,
 96755: 0.006880366659086047,
 272218: 0.017452233448569058,
 285591: 0.009459113655451611,
 157702: 0.007866388026631622,
 140436: 0.012478370414871954,
 236478: 0.006523074607143577,
 201971: 0.018468876

In [110]:
i2i_sim[303292]

{303331: 0.024100940150752093,
 303597: 0.21497370732009732,
 305094: 0.07325821764128491,
 62464: 0.03104482873740403,
 301812: 0.44755904483465825,
 285435: 0.003782937070997355,
 289126: 0.0027417300487650033,
 277492: 0.0020677438315884855,
 283392: 0.0034356359323810368,
 283996: 0.011633910814789802,
 303350: 0.2240822319971309,
 307779: 0.008612435133046627,
 97530: 0.009591226094917502,
 123289: 0.003446983241071964,
 157478: 0.005382436677636408,
 42883: 0.004499858900717249,
 95977: 0.0035358666011787764,
 62764: 0.004059208624192752,
 303418: 0.12518243045922517,
 304040: 0.1090508242282041,
 304640: 0.06211747716024042,
 304705: 0.06741080051916089,
 304526: 0.09725295502285797,
 62463: 0.009207727205905208,
 206785: 0.003742794360269522,
 202355: 0.0017992906249709912,
 313431: 0.019951833810979286,
 303819: 0.5201466684542275,
 304310: 0.04595316759113198,
 213877: 0.021004906093656375,
 184003: 0.047775580797626345,
 209348: 0.008499267988638662,
 289003: 0.0017522686148

In [111]:
i2i_sim[303350]

{293301: 0.006942089432751088,
 156560: 0.01755034097741752,
 233717: 0.013285113849722315,
 237612: 0.009622947828543369,
 161801: 0.007178636605949193,
 272513: 0.044730092122223945,
 50504: 0.010763012263460988,
 71076: 0.04086142299456134,
 266008: 0.017017139099483148,
 129434: 0.007029038660524041,
 57936: 0.043605148516410605,
 233478: 0.0357654646939914,
 160940: 0.0037685043501531016,
 208430: 0.005492912861237238,
 203890: 0.004024061457798026,
 303565: 0.011098190835074174,
 297639: 0.020966152108340658,
 233658: 0.005604555350630356,
 286310: 0.00046696339995190654,
 285591: 0.0007115245853291634,
 286108: 0.005162590944680541,
 285849: 0.0003547584175636394,
 293050: 0.0011748838776046678,
 292454: 0.005175045157036025,
 282964: 0.007816018083875885,
 277067: 0.000844172369706379,
 276970: 0.014422317729321045,
 277491: 0.003958314498846768,
 283776: 0.002723535636901423,
 338339: 0.0016870645303761773,
 342471: 0.013432206080330215,
 336254: 0.0017100433624307847,
 336934

In [112]:
i2i_sim[303967]

{303533: 0.11799743634485092,
 304664: 0.11810795891071033,
 304526: 0.2210358014051354,
 157702: 0.014915343805048734,
 133160: 0.004255528510425723,
 257291: 0.00404392752570355,
 159762: 0.002543336696320454,
 285719: 0.003438981777845023,
 289003: 0.003574337392066166,
 283009: 0.0054858898407921216,
 111587: 0.09255951642868465,
 47799: 0.08147008727129493,
 47718: 0.01005688591013183,
 48868: 0.053569636991816,
 48466: 0.0574969679249702,
 50494: 0.004910515371491351,
 19382: 0.024263921874681117,
 15209: 0.0157355121191182,
 234195: 0.003032817203090889,
 234173: 0.007756554924933972,
 256026: 0.003533045015545745,
 255747: 0.005814878224437332,
 237203: 0.0065221503805811985,
 182303: 0.00690239516815325,
 198317: 0.0055219161345226005,
 198925: 0.0030304025739078966,
 286735: 0.026722395666776852,
 303535: 0.030652690063365646,
 303845: 0.05755308684593788,
 303808: 0.05155952318288908,
 304093: 0.021028828205137785,
 304017: 0.4224030790029174,
 304344: 0.3146824297955055,
 3

In [113]:
i2i_sim[304017]

{324823: 0.046336944368902094,
 337143: 0.00920633970206016,
 303597: 0.20035855309679612,
 14748: 0.04778745387659583,
 87228: 0.0169557568977838,
 87192: 0.012378492654586806,
 304226: 0.306360807125935,
 304735: 0.3787920992025555,
 286310: 0.008517790459923557,
 285591: 0.0007160741469406076,
 286108: 0.006706859609725727,
 285849: 0.00035532099132540296,
 293050: 0.00023327965945559536,
 292454: 0.005208134929714855,
 282964: 0.0006681490336054519,
 277067: 0.0008495700949373088,
 276970: 0.002092430465417191,
 277491: 0.0006781425689732885,
 283776: 0.0027409502047695384,
 338339: 0.001655262330335861,
 342471: 0.013518092991900255,
 336254: 0.001978619166143761,
 336934: 0.006248701134862791,
 303350: 0.28658078548202437,
 303180: 0.015663732427992322,
 307149: 0.07906853774427791,
 331116: 0.003311676945911134,
 123909: 0.002480187497335708,
 123757: 0.0025010537345958417,
 111210: 0.0037940163993169106,
 111043: 0.0018280989781218136,
 140528: 0.003341569620266332,
 133160: 0.

In [114]:
i2i_sim[324823]

{292390: 0.0018128749823372713,
 337143: 0.00669601198917347,
 129434: 0.04769148642817574,
 31520: 0.032394691772507964,
 83534: 0.011980386933188071,
 95977: 0.014488876740121634,
 217810: 0.0052737997740692055,
 214474: 0.005518986917743967,
 272143: 0.400691061394807,
 235872: 0.006500974737779746,
 162809: 0.0008838518106873727,
 160940: 0.026816321514497584,
 288977: 0.004290425154896231,
 288325: 0.023119333703602807,
 289186: 0.014810924409286412,
 289126: 0.016605812976929396,
 303180: 0.00688929942734777,
 97546: 0.007713948574208814,
 120967: 0.009149466767165217,
 119534: 0.013529902014628737,
 9308: 0.011284128205150575,
 9126: 0.017732145539486037,
 96321: 0.002274689104049096,
 215993: 0.012387844103048854,
 264458: 0.005872684228875365,
 182883: 0.0015404266778963709,
 304017: 0.03518067958520737,
 293114: 0.0292684479447781,
 283238: 0.012407219411447488,
 337735: 0.014969676579156125,
 313365: 0.0005359417334513507,
 312305: 0.0020489767868059994,
 312619: 0.000882830